Steps:
- Enable data augmentation, and precompute=True
- Use lr_find() to finmd highest learning rate where loss is clearly improving
- Train last layer from pre-computed activations for 1-2 epochs
- Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
- Unfreeze all layers
- Set earlier layers to 3x-10x lower learning rate than next higher layer
- Use lr_find() again
- Train full network with cycle=mult=2 until over-fitting

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sklearn import metrics
from fastai.conv_learner import *

C:\tools\Anaconda3\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = 'data/dogcatbreed/'

In [4]:
bs=64
f_model = resnet34

In [5]:
# Data preparation steps if you are using Crestle:

os.makedirs('data/dogcatbreed/models', exist_ok=True)
os.makedirs('/cache/dogcatbreed/tmp', exist_ok=True)

In [6]:
label_csv= f'{PATH}train.csv'
n = len(list(open(label_csv))) - 1 #count row in file, exclude the header row
val_idxs = get_cv_idxs(n) # function choses random 20% data for valdation set

In [7]:
def get_data(sz, bs):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test', bs=bs)

In [8]:
sz=64
data = get_data(sz,bs)

In [9]:
data = data.resize(int(sz*1.3), 'tmp')

In [10]:
learn = ConvLearner.pretrained(f_model, data, ps=0.5) #ps sets the dropout rate

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [13]:
lr = 1e-2

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [14]:
lrs = np.array([lr/9,lr/3,lr])

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save(f'{sz}')

In [ ]:
learn.sched.plot_loss()

In [ ]:
sz=128

In [ ]:
learn.set_data(get_data(sz,bs))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

In [11]:
sz=128
learn.load(f'{sz}')
sz=256

In [15]:
learn.set_data(get_data(sz,16))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.664571   0.343478   0.890402  
    1      0.614456   0.345738   0.886998                                                                              
    2      0.560548   0.31998    0.890402                                                                              
    3      0.61169    0.335146   0.892444                                                                              
    4      0.567239   0.308664   0.903336                                                                              
    5      0.568852   0.307665   0.89857                                                                               
    6      0.520257   0.29815    0.904697                                                                              



[array([0.29815]), 0.9046970729603907]

In [16]:
learn.unfreeze()
learn.bn_freeze(True)
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.995478   0.503094   0.840027  
    1      1.290986   0.727568   0.758339                                                                              
    2      0.578011   0.397641   0.862491                                                                              
    3      1.22854    0.909402   0.707284                                                                              
    4      0.807114   0.557884   0.827774                                                                              
    5      0.489458   0.400343   0.872022                                                                              
    6      0.397656   0.353152   0.88904                                                                               



In [17]:
sz=299
learn.set_data(get_data(sz,16))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.414596   0.347009   0.882914  
    1      0.348801   0.339398   0.891763                                                                              
    2      0.346371   0.343235   0.890402                                                                              
    3      0.419045   0.353102   0.885636                                                                              
    4      0.365495   0.348788   0.87951                                                                               
    5      0.330214   0.343765   0.885636                                                                              
    6      0.296161   0.342165   0.883594                                                                              



[array([0.34217]), 0.8835942819866702]

In [18]:
learn.unfreeze()
learn.bn_freeze(True)
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.494519   0.408743   0.868618  
    1      0.764036   0.562647   0.810756                                                                              
    2      0.356399   0.351099   0.882914                                                                              
    3      0.871881   0.628604   0.795099                                                                              
    4      0.605914   0.438012   0.85228                                                                               
    5      0.370129   0.336898   0.885636                                                                              
    6      0.288081   0.318677   0.901293                                                                              



In [19]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

(0.8951667801225324, 0.31617706290602854)

# Run on test data now and create a submission

In [20]:
log_preds,y = learn.TTA(is_test=True) #use test dataset not validation dataset
probs = np.mean(np.exp(log_preds),0)

In [21]:
df = pd.DataFrame(probs)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])
df.head()

,id,abyssinian,american_bulldog,american_pit_bull_terrier,basset_hound,beagle,bengal,birman,bombay,boxer,...,russian_blue,saint_bernard,samoyed,scottish_terrier,shiba_inu,siamese,sphynx,staffordshire_bull_terrier,wheaten_terrier,yorkshire_terrier
0,0006246bf61f5801e125c305d2ed25e2,2.681797e-04,2.959661e-03,2.162147e-03,5.902693e-04,8.331216e-04,3.257815e-04,2.498083e-03,0.000353,1.220311e-02,...,0.001132,2.347799e-03,1.992941e-04,3.260463e-04,2.118262e-03,8.098780e-02,2.141491e-04,6.761466e-03,4.934667e-04,1.267245e-04
1,001e73b6915f4128065470cdd3766fcd,6.488925e-05,1.721284e-07,1.599076e-08,5.144778e-08,5.711307e-09,5.432115e-03,1.974599e-08,0.000010,4.102268e-08,...,0.000197,4.511386e-09,8.563491e-08,9.062567e-08,4.008244e-08,6.193928e-08,2.614930e-05,1.698273e-08,1.101904e-07,5.389499e-08
2,002d5910de023eddce8358edf169c07f,1.344918e-06,5.133011e-04,1.534123e-05,6.781430e-07,9.542901e-06,1.196019e-06,5.962776e-06,0.000002,2.877464e-06,...,0.000056,1.439581e-04,9.224401e-01,7.171273e-06,3.711433e-03,1.696454e-06,2.421740e-06,2.817564e-04,3.323782e-05,6.531671e-08
3,003b7dec4fe65ec846854c7597617a23,6.620718e-07,1.225802e-04,6.207456e-02,6.473377e-05,9.905028e-06,1.093506e-06,9.512871e-09,0.000005,7.516094e-04,...,0.000028,2.004240e-07,6.014229e-08,1.088908e-06,1.469549e-06,2.830627e-07,8.495577e-06,1.403855e-01,1.845076e-07,1.013325e-07
4,004955e30603d8c06c1ff917736eb418,9.827052e-07,8.363490e-10,1.925780e-09,5.034797e-09,4.374346e-09,2.123180e-07,5.158247e-09,0.999656,3.182263e-08,...,0.000072,2.177004e-10,2.074316e-09,9.892935e-07,2.390029e-09,1.666318e-07,2.348233e-08,9.611413e-09,9.252313e-10,2.335749e-08


In [22]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}fs1-subm1.gz', compression='gzip', index=False)
FileLink(f'{SUBM}fs1-subm1.gz')

C:\Users\nissan.dookeran\Documents\Github\fastai\courses\dl1\data\dogcatbreed\subm\fs1-subm1.gz